In [1]:
#set environment variables after creating environment from the yaml file provided
!setup.bat


(ryzenai-1) E:\experiments\ryzenai_chatbot\Chatbot-with-RyzenAI-1.0>SET PWD=E:\experiments\ryzenai_chatbot\Chatbot-with-RyzenAI-1.0\ 

(ryzenai-1) E:\experiments\ryzenai_chatbot\Chatbot-with-RyzenAI-1.0>SET THIRD_PARTY=E:\experiments\ryzenai_chatbot\Chatbot-with-RyzenAI-1.0\\third_party 

(ryzenai-1) E:\experiments\ryzenai_chatbot\Chatbot-with-RyzenAI-1.0>SET TVM_LIBRARY_PATH=E:\experiments\ryzenai_chatbot\Chatbot-with-RyzenAI-1.0\\third_party\lib;E:\experiments\ryzenai_chatbot\Chatbot-with-RyzenAI-1.0\\third_party\bin 

(ryzenai-1) E:\experiments\ryzenai_chatbot\Chatbot-with-RyzenAI-1.0>SET PATH=C:\Users\MicroZaib\miniconda3\envs\ryzenai-1;C:\Users\MicroZaib\miniconda3\envs\ryzenai-1\Library\mingw-w64\bin;C:\Users\MicroZaib\miniconda3\envs\ryzenai-1\Library\usr\bin;C:\Users\MicroZaib\miniconda3\envs\ryzenai-1\Library\bin;C:\Users\MicroZaib\miniconda3\envs\ryzenai-1\Scripts;C:\Users\MicroZaib\miniconda3\envs\ryzenai-1\bin;C:\Users\MicroZaib\miniconda3\condabin;C:\Windows\system32;C:\W

In [2]:
#Add required system paths
import sys
sys.path.append("../ext") 
sys.path.append("../smoothquant_package") 
sys.path.append("./gradio_app") 

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import logging
import time
import gc
import os
from optimum.onnxruntime import ORTModelForCausalLM
from transformers import set_seed, AutoTokenizer, OPTForCausalLM
import pathlib
# import smooth
import torch
import random 
import string
import gradio as gr
import time
from modeling_ort_amd import ORTModelForCausalLM
from smoothquant_package.smooth import smooth_lm


In [5]:
# function to define model path where downloaded pretrained model would be saved
model_id = "facebook/opt-1.3b" #path to pretrained model on huggingface hub
def model_path_fp32(model_id):
    model_name = model_id.rsplit('/')[-1]
    out_dir = "./%s_pretrained_fp32"%model_name
    model_path_fp32 = out_dir  
    return model_path_fp32

model_path_fp32(model_id)

'./opt-1.3b_pretrained_fp32'

In [6]:
# download and save model from huggingface model hub
def download_model(model_id):
    """ Downloads and saves a pretratined model from (e.g. HF) to the specified dir
    Args: 
    model_path_pt - pretrained model path (e.g. facebook/opt-1.3b) 
    Output: fp-32 model saved in specified dir """
    
    model_name = model_id.rsplit('/')[-1] #extract model name from model path pretrained using python string methods
    model = OPTForCausalLM.from_pretrained(model_id)
#specify a dir for saving the model
    out_dir = "./%s_pretrained_fp32"%model_name
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    model.save_pretrained(out_dir)
    # print(out_dir)

download_model(model_id)



- Create activation channel scales for the downloaded modelfor smooth quantization
- To be incorporated later from https://github.com/mit-han-lab/smoothquant
- Using downloaded act_scales in this example

In [7]:
#quantize the downloaded fp32 model to int8 quantized model after first smooth-quantizing
def model_quant(model_path_fp32, model_id):
    """ Quantizes and saves the downloaded fp32 pretratined model to the specified dir, first as smooth quant and then as int8 quantized model
    Args: 
    model_path_fp32 - downloaded fp32 model path  
    model_path_pt - pretrained model path (e.g. facebook/opt-1.3b)
    Output: smooth quant model saved in specified dir and int-8 quantized model saved in specified dir """
    
    path = model_path_fp32(model_id)
    model_name = model_id.rsplit('/')[-1]
    if not os.path.exists(path):
            print(f"Pretrained fp32 model not found, exiting..")
            exit(1)
    model = OPTForCausalLM.from_pretrained(path)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model.tokenizer = tokenizer 
     # get actuvation scales for smooth quantization
    model_act = "%s.pt"%model_name
    act_scales = torch.load("./ext/smoothquant/act_scales/" + model_act)
    # smooth quantize
    smooth_lm(model, act_scales, 0.5)
    print(model)
    # initialize with random prompt
    prompt = ''.join(random.choices(string.ascii_lowercase + " ", k=model.config.max_position_embeddings))
    #inputs = tokenizer(prompt, return_tensors="pt")  # takes a lot of time
    inputs = tokenizer("What is meaning of life", return_tensors="pt") 
    print(f"inputs: {inputs}")
    print(f"inputs.input_ids: {inputs.input_ids}")
    for key in inputs.keys():
        print(inputs[key].shape)
        print(inputs[key])
    model_out = model(inputs.input_ids)
    print(f"{(model_out.logits.shape)=}")
    out_dir = "./%s_smoothquant"%model_name
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    # model.save_pretrained(out_dir+"/model_onnx")
    model.save_pretrained(os.path.join(out_dir, "model_onnx"))
    print(f"Saving Smooth Quant fp32 model...\n ")

    print(f"Quantizing model with Optimum...\n ")
    #proc = subprocess.Popen('cmd.exe', stdin = subprocess.PIPE, stdout = subprocess.PIPE)
    # os.system('optimum-cli export onnx -m out_dir\model_onnx --task text-generation-with-past out_dir\model_onnx_int8  --framework pt --no-post-process')
    os.system(f'optimum-cli export onnx -m {os.path.join(out_dir, "model_onnx")} --task text-generation-with-past {os.path.join(out_dir, "model_onnx_int8")} --framework pt --no-post-process')

    print(f"Saving quantized int8 model ...\n ")  


In [8]:
model_quant(model_path_fp32, model_id="facebook/opt-1.3b")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
          (final_layer_norm): LayerN

In [9]:
config_file_path = "./vaip_config.json"
providers = ["VitisAIExecutionProvider", "CPUExecutionProvider"]
provider_options = [{'config_file': str(config_file_path)}, {}]

In [10]:
def generate_text(Input_text, max_output_token, model_id = "facebook/opt-1.3b", p_type = 0, p_options = 0):
    model_name = model_id.rsplit('/')[-1]
    out_dir = "./%s_smoothquant"%model_name    
    onnx_model_path = out_dir+"/model_onnx_int8"
    model = ORTModelForCausalLM.from_pretrained(onnx_model_path, provider = providers[p_type], provider_options = provider_options[p_options])
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    inputs = tokenizer(Input_text, return_tensors="pt") 
    s = time.perf_counter()
    outputs_tkn = model.generate(inputs.input_ids, max_length=max_output_token, use_cache=True, do_sample=False)
    e = time.perf_counter() - s
    outputs_tkn_len = outputs_tkn.shape[1]
    outputs = tokenizer.batch_decode(outputs_tkn,
                                    skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=False)[0]
    print(outputs)
    yield outputs, "tkn/sec: " + str(outputs_tkn_len/e)

In [11]:
gr.Interface(
    fn=generate_text,
    inputs=["text", gr.Slider(minimum=32, maximum=256, value=32, step = 32)],
    outputs=["text", "text"],
    title="OPT Chatbot on Ryzen AI with NPU",
    description="Simple Chatbot on Ryzen AI Laptop using the NPU",
    concurrency_limit=4
    #).queue(concurrency_count=2).launch(server_name="localhost", server_port=1234)
    ).queue().launch(server_name="localhost", server_port=1235)

Running on local URL:  http://localhost:1235

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 4.8.0, however version 4.29.0 is available, please upgrade.
--------
Couldn't find dll file  libGemmQnnAie_8x2048_2048x2048.dll,


CRITICAL:root:[PROFILE][ORTAMD] model_decoder_forward 121855200 3116700 1587800
CRITICAL:root:[PROFILE][ORTAMD] model_decoder_forward 107018800 190000 147600
CRITICAL:root:[PROFILE][ORTAMD] model_decoder_forward 104910400 226400 137700
CRITICAL:root:[PROFILE][ORTAMD] model_decoder_forward 100851100 200800 142400
CRITICAL:root:[PROFILE][ORTAMD] model_decoder_forward 101198200 191800 155400
CRITICAL:root:[PROFILE][ORTAMD] model_decoder_forward 101519500 200200 145400
CRITICAL:root:[PROFILE][ORTAMD] model_decoder_forward 103117300 202800 147000
CRITICAL:root:[PROFILE][ORTAMD] model_decoder_forward 105751900 236200 145700
CRITICAL:root:[PROFILE][ORTAMD] model_decoder_forward 100429000 199200 139700
CRITICAL:root:[PROFILE][ORTAMD] model_decoder_forward 104795400 208500 148600
CRITICAL:root:[PROFILE][ORTAMD] model_decoder_forward 101023900 188000 148500
CRITICAL:root:[PROFILE][ORTAMD] model_decoder_forward 101375300 204600 136600
CRITICAL:root:[PROFILE][ORTAMD] model_decoder_forward 10051830

Where is India?AprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprAprApr


In [17]:
import torch




In [21]:
import torchaudio


OSError: [WinError 127] The specified procedure could not be found